In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('ActivityLog.csv', delimiter=',', skiprows=range(28), header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.670083,-0.670025,-0.792625,-0.942348,-2.045735,-1.870625,-2.466390,-0.800682,-0.410636,0.684675,...,-0.769320,-0.397894,0.855070,-0.104602,-0.300502,-0.828248,0.206412,-0.738789,-0.005197,3
1,-0.861629,-1.067786,-1.004663,-1.192020,-1.691732,-1.873450,-2.483531,-1.100105,-0.334030,0.660999,...,-0.899660,-1.194255,-0.134053,-0.158409,-0.321210,-1.284549,0.388559,-0.727133,-0.028941,3
2,-1.120217,-1.243872,-1.088129,-1.321978,-1.568390,-1.843448,-2.466832,-1.230669,-0.312127,0.660937,...,-1.060589,-1.274840,-0.126441,-0.207583,-0.256539,-1.379049,0.374665,-0.731674,-0.029147,3
3,-1.148949,-1.275784,-1.098498,-1.334028,-1.570453,-1.812270,-2.465487,-1.217213,-0.315851,0.660845,...,-1.077879,-1.355909,-0.080027,-0.234463,-0.237908,-1.422561,0.379111,-0.732445,-0.029891,3
4,-1.152780,-1.275784,-1.097461,-1.330954,-1.569405,-1.810911,-2.467587,-1.191753,-0.315808,0.660839,...,-1.080539,-1.358767,-0.077900,-0.241134,-0.246843,-1.428392,0.379117,-0.732428,-0.030026,3


In [5]:
len(df)

1143

In [7]:
dataset = df.values
len(dataset)

1143

In [10]:
dataset.shape
type(dataset), type(df)

(numpy.ndarray, pandas.core.frame.DataFrame)

In [12]:
X = dataset[:,:24].astype(float)
Y = dataset[:,24]

In [21]:
X.shape, Y.shape

((1143, 24), (1143,))

In [23]:
Yt = Y.reshape(-1,1)
Yt.shape

(1143, 1)

In [18]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)
# print(Y)
# print(encoded_Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)
enc = OneHotEncoder()
enc.fit(Y.reshape(1,-1))
enc.n_values_

array([4, 4, 4, ..., 2, 2, 2])

In [19]:
enc.feature_indices_


array([   0,    4,    8, ..., 6516, 6518, 6520])

In [20]:
Y_test = enc.transform(Y.reshape(1,-1))
Y_test.shape

(1, 1143)

In [25]:
ohe = OneHotEncoder()
Ytt = ohe.fit_transform(Yt)
print(Ytt.shape)

(1143, 8)


In [34]:
print(Ytt[:5])

  (0, 3)	1.0
  (1, 3)	1.0
  (2, 3)	1.0
  (3, 3)	1.0
  (4, 3)	1.0


In [35]:
print(X[:5])

[[-0.67008273 -0.67002541 -0.79262519 -0.94234768 -2.04573465 -1.87062467
  -2.4663898  -0.8006816  -0.41063581  0.68467463  0.43051885  0.08926412
  -0.39963514 -0.76629388 -1.04328953 -0.76932046 -0.39789395  0.85506963
  -0.10460162 -0.30050204 -0.8282483   0.20641181 -0.73878926 -0.00519707]
 [-0.86162909 -1.06778633 -1.00466254 -1.19201971 -1.69173191 -1.87344973
  -2.48353061 -1.10010543 -0.33402968  0.66099898  0.20462101  0.14537389
  -0.44750585 -1.19639224 -1.19668494 -0.89965964 -1.19425462 -0.1340528
  -0.15840925 -0.32121047 -1.28454926  0.38855861 -0.72713313 -0.02894112]
 [-1.12021668 -1.24387247 -1.08812918 -1.32197778 -1.56838984 -1.84344819
  -2.46683244 -1.23066911 -0.31212664  0.66093721  0.20757578  0.05890183
  -0.76954518 -1.24787847 -1.2653933  -1.06058865 -1.27483962 -0.12644102
  -0.20758273 -0.25653851 -1.37904917  0.37466526 -0.73167366 -0.02914659]
 [-1.14894864 -1.27578449 -1.09849772 -1.33402825 -1.57045347 -1.81226997
  -2.46548721 -1.21721264 -0.3158513

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [42]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [45]:
print(type(Ytt))
Ytt = Ytt.todense()
print(type(Ytt))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.matrixlib.defmatrix.matrix'>


In [46]:
print(Ytt.shape)

(1143, 8)


In [52]:
kf = KFold(n_splits=5)
# print(list(kf.split(X)))
# print(list(kf.split(Y)))
for train, test in kf.split(X):
    XTrain, XTest, YTrain, YTest = X[train], X[test], Ytt[train], Ytt[test]
    clf = RandomForestClassifier(n_estimators=500, random_state=seed, )
    clf = clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    print((100.0 * score)/(YPred.size))

96.2882096069869
96.34279475982532
93.83187772925764
95.12061403508773
97.14912280701755


In [50]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
# print(list(kf.split(X)))
# print(list(kf.split(Y)))
for train, test in skf.split(X, Ytt):
    XTrain, XTest, YTrain, YTest = X[train], X[test], Ytt[train], Ytt[test]
    clf = RandomForestClassifier(n_estimators=10, random_state=seed, )
    clf = clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    print((100.0 * score)/(YPred.size))

IndexError: too many indices for array